# Webscrapping de l'indice VIX

## Obtenir la bonne page HTML avec Selenium

Avant de commencer le webscrapping, nous devons effectuer quelques modifications dans le code HMTML de base de la pgae web. Ainsi, pour obtenir le jeu de données entier, nous allons utiliser la librairie **selenium** qui nous permettra d'obtenir la page HTML avec toutes les données.

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC


In [69]:
import time
driver = webdriver.Chrome(ChromeDriverManager().install())#,options=chrome_options)
driver.get('https://fr.investing.com/indices/cac-40-vix-historical-data')
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()

#elements =driver.find_elements(By.CSS_SELECTOR,'css-1545rns-container')
dropdown =driver.find_element("id",'history-timeframe-selector')
dropdown.click() # on ouvre le dropdown

# on cherche les différentes options présentes dans le dropdown et on clique sur celle qu'on veut
driver.find_element("id","react-select-2-option-0").click() # 0 -> journalier, 1 -> Hebdomadaire, 2 -> pour Mensuel 


calendar=driver.find_element(By.CLASS_NAME,"historical-data_history-date-picker-wrapper__dDOuq")
calendar.click()

date_input = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div/div[2]/main/div/div[5]/div/div/div[2]/div[2]/div[2]/div[2]/div[1]/div/div[1]/input')
date_input.clear()
date_input.send_keys("3004-08-2005") #date à laquelle on souhaite commencer (marche pas mais permet de donner tout l'historique pour l'instant)


validate=driver.find_element(By.CLASS_NAME,"HistoryDatePicker_arrow-icon__NwxN4")
validate.click()

#driver.find_element(By.CLASS_NAME,"inv-button signup_close__MFV87")
#WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,"inv-button signup_close__MFV87"))).click() # on attend 10 secondes avant de break en l'abscence du bouton


time.sleep(6)

webpage=driver.page_source # on stock le code html de la page où toutes le données sont chargées
#driver.quit()  



C:\Users\elelo\AppData\Local\Temp/ipykernel_16136/3996943661.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())#,options=chrome_options)


## From HTML to Dataframe :

Maintenant que nous avons le code HTML de toutes les données, nous allons décortiquer la page grâce à la librairie BeautifulSoup.

In [70]:
import bs4
import pandas

In [71]:
# utiliser le package BeautifulSoup qui "comprend" les balises 

page = bs4.BeautifulSoup(webpage, "lxml")

print(page)

<html class="html" dir="ltr" lang="fr"><head><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-PGT3R3D&amp;l=dataLayerGTMParams"></script><script async="" gtm="GTM-PGT3R3D" src="https://www.googleoptimize.com/optimize.js?id=GTM-WW3TL3X&amp;l=dataLayerGTMParams" type="text/javascript"></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-C4NDLGKVMK&amp;l=dataLayerGTMParams&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script>
                window.geo_country = 'FR';
                window.geo_state = 'IDF';
            </script>
<script charset="UTF-8" data-domain-script="265d5ac2-7d0c-4675-bf4d-421a61fb482e" src="https://cookie-cdn.cookiepro.com/scripttemplates/otSDKStub.js" type="text/javascript"></script>
<meta charset="utf-8"/><meta content="initial-scale=1.0, width

In [72]:
# on identifie le tableau des indices vix : c'est le premier qui a cette classe "datatable_table__D_jso datatable_table--border__B_zW0 datatable_table--mobile-basic__W2ilt datatable_table--freeze-column__7YoIE"
tableau_indice = page.find('table', {'class' : 'datatable_table__D_jso datatable_table--border__B_zW0 datatable_table--mobile-basic__W2ilt datatable_table--freeze-column__7YoIE'})


In [73]:
table_body = tableau_indice.find('tbody')

# on recherche toutes les lignes du tableau avec la balise "tr"
rows = table_body.find_all('tr')

# on obtient une liste où chaque élément est une des lignes du tableau



In [74]:
cols = rows[0].find_all('td')
print(cols[0])
print(cols[0].text.strip())

<td class="datatable_cell__0y0eu font-bold"><time datetime="20/04/2022">20/04/2022</time></td>
20/04/2022


In [75]:
for ele in cols : 
    print(ele.text.strip())

20/04/2022
18,96
18,96
18,96
18,96

0.00%


Voici le dictionnaire de tous les indices VIX :

In [76]:
dico_vix = dict()
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols) > 0 : 
        dico_vix[cols[0]] = cols[1:]
dico_vix

{'20/04/2022': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '04/04/2022': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '22/02/2022': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '02/02/2022': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '27/01/2022': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '30/12/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '28/12/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '23/12/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '22/12/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '21/12/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '20/12/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '17/12/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '08/12/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '09/11/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '28/10/2021': ['18,96', '18,96', '18,96', '18,96', '', '0.00%'],
 '11/10/20

In [78]:
data_vix = pandas.DataFrame.from_dict(dico_vix,orient='index')
data_vix.head()

,0,1,2,3,4,5
20/04/2022,"18,96","18,96","18,96","18,96",,0.00%
04/04/2022,"18,96","18,96","18,96","18,96",,0.00%
22/02/2022,"18,96","18,96","18,96","18,96",,0.00%
02/02/2022,"18,96","18,96","18,96","18,96",,0.00%
27/01/2022,"18,96","18,96","18,96","18,96",,0.00%


In [81]:
table_head=page.find("thead",{'class': 'datatable_head__ysELt'})
col_name_head=[]
for x in table_head.find_all("tr"):
    
    for y in x.find_all('th'):
        col_name_head.append(y.text)
col_name_head

['', 'Signe', 'Marché', 'Devise', '']

In [80]:
data_vix.columns=col_name_head[1:]

ValueError: Length mismatch: Expected axis has 6 elements, new values have 4 elements

In [49]:
data_vix

,0,1,2,3,4,5
08/08/2019,"17,51","19,28","21,03","15,36",,-12.01%
07/08/2019,"19,90","20,32","46,39","16,52",,-2.07%
06/08/2019,"20,32","17,80","20,52","17,59",,+14.09%
05/08/2019,"17,81","17,51","21,36","15,10",,+1.77%
02/08/2019,"17,50","13,42","46,95","13,42",,+30.50%
...,...,...,...,...,...,...
10/01/2000,"25,85","25,85","25,85","25,85",,+0.70%
07/01/2000,"25,67","25,67","25,67","25,67",,-6.42%
06/01/2000,"27,43","27,43","27,43","27,43",,-5.12%
05/01/2000,"28,91","28,91","28,91","28,91",,-1.70%


In [89]:
from selenium.webdriver.support.ui import Select
import time

# Les deux lignes suivantes pemettent d'empêcher l'apparition de pop up (cookies, agreement...) sur la page
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.cookies": 2})

driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
driver.get('https://www.boursorama.com/bourse/indices/cours/historique/1cVIX')
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))).click()
#WebDriverWait(driver, 10).until(EC.visibility_of((By.ID, "historic_search[duration]")))

#select = Select(driver.find_element(By.ID,'historic_search[duration]'))
#select.select_by_visible_text("3 ans")


#element=WebDriverWait(driver,30).until(EC.presence_of_element_located((By.CSS_SELECTOR,"option[value='1']")))
#select=Select(driver.find_element(By.ID,'historic_search_period'))

#select.select_by_visible_text("Hebdo")

from selenium.common.exceptions import TimeoutException

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "option[value='7']"))
    )
    print("Option loaded")
except TimeoutException:
    print("Time exceeded!")

select=Select(driver.find_element(By.ID,'historic_search_period'))

select.select_by_visible_text("Hebdo")




C:\Users\elelo\AppData\Local\Temp/ipykernel_17784/3602946800.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)


Option loaded


ElementNotInteractableException: Message: element not interactable: Element is not currently visible and may not be manipulated
  (Session info: chrome=107.0.5304.88)
Stacktrace:
Backtrace:
	Ordinal0 [0x009FACD3+2075859]
	Ordinal0 [0x0098EE61+1633889]
	Ordinal0 [0x0088B7BD+571325]
	Ordinal0 [0x0088E374+582516]
	Ordinal0 [0x0088E225+582181]
	Ordinal0 [0x0088E4C0+582848]
	Ordinal0 [0x008BE7BE+780222]
	Ordinal0 [0x008B2166+729446]
	Ordinal0 [0x008D731C+881436]
	Ordinal0 [0x008B15BF+726463]
	Ordinal0 [0x008D7534+881972]
	Ordinal0 [0x008EB56A+963946]
	Ordinal0 [0x008D7136+880950]
	Ordinal0 [0x008AFEFD+720637]
	Ordinal0 [0x008B0F3F+724799]
	GetHandleVerifier [0x00CAEED2+2769538]
	GetHandleVerifier [0x00CA0D95+2711877]
	GetHandleVerifier [0x00A8A03A+521194]
	GetHandleVerifier [0x00A88DA0+516432]
	Ordinal0 [0x0099682C+1665068]
	Ordinal0 [0x0099B128+1683752]
	Ordinal0 [0x0099B215+1683989]
	Ordinal0 [0x009A6484+1729668]
	BaseThreadInitThunk [0x7512FA29+25]
	RtlGetAppContainerNamedObjectPath [0x7